In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter
from scipy.signal import peak_widths
from matplotlib.gridspec import GridSpec
import csv
import os
import ipympl
import pprint

class LaserData:
    def __init__(self,path):
        self.wavelenghts = np.array([229,233,237,241,245,249,253]) + 780
        with open(path) as file:
            self.file = file
            self.file = csv.reader(self.file, delimiter='\t')
            self.time =[]
            self.piezo_voltage = []
            self.fabry_perot = []
            self.probe_laser = []
            self.laser_voltage = []
            for line in self.file:
                self.time.append(float(line[0]))
                self.piezo_voltage.append(5 * float(line[1]))
                self.fabry_perot.append(float(line[2]))
                self.probe_laser.append(float(line[3]))
                self.laser_voltage.append(float(line[4]))
            self.time = np.array(self.time)
            self.piezo_voltage = np.array(self.piezo_voltage)
            self.fabry_perot = np.array(self.fabry_perot)
            self.probe_laser = np.array(self.probe_laser)
    
    def plot(self,with_peaks = False):
        fig = plt.figure(figsize=(11,6))
        gs = GridSpec(8,5)
        fig1 = fig.add_subplot(gs[:,:])
        fig1.plot(self.piezo_voltage,self.fabry_perot,'-')
        #fig1.plot(self.time,self.piezo_voltage)
        if with_peaks:
            peaks,_ = find_peaks(self.fabry_perot,prominence=0.3,height=0.8)
            print(len(peaks))
            for peak in peaks:
                fig1.plot(self.piezo_voltage[peak],self.fabry_perot[peak],'go')
        fig1.set_xlabel('Zeit in s',fontsize = 15)
        fig1.set_ylabel('Spannung in V',fontsize = 15)

    def slice_data(self, idx_start, idx_stop, filter=False):
        # deletes the data outside the given index_range
        self.time = self.time[idx_start:idx_stop] - self.time[idx_start]
        self.piezo_voltage = self.piezo_voltage[idx_start:idx_stop]
        self.fabry_perot = self.fabry_perot[idx_start:idx_stop]
        self.probe_laser = self.probe_laser[idx_start:idx_stop]
        self.laser_voltage = self.laser_voltage[idx_start:idx_stop]

        if filter:
            self.piezo_voltage = savgol_filter(self.piezo_voltage, 1000, 1)
            self.fabry_perot = savgol_filter(self.fabry_perot, 50, 6)
            self.probe_laser = savgol_filter(self.probe_laser, 50, 6)

laser = LaserData('Data/spektrum_dopplerfrei_mit_pump_freq4.dat') 

# Nicht dopplerfrei

In [ ]:
def piezo2wavelength(voltage, freq=3):
    # returns the corresponding wavelength fot given Piezo voltage
    # by given freqency setting (1, 3, 4, 6). Default is freq=3
    #korrespondierende Wellenlängen
    wavelengths_nm = np.array([229, 233, 237, 241, 245, 249]) * 10**(-3) + 780

    laser = LaserData(f'Data/spektrum_dopplerfrei_ohne_pump_freq{freq}.dat')
    #starting time
    idx_start = np.argmax(laser.piezo_voltage)
    idx_stop  = np.argmin(laser.piezo_voltage)

    laser.slice_data(idx_start, idx_stop, filter=True)

    peaks, _ = find_peaks(laser.fabry_perot,prominence=0.05,height=0.8)

    if freq==1:
        x, y = laser.piezo_voltage[peaks[1:7]][::-1], wavelengths_nm[::-1]
    else:
        x, y = laser.piezo_voltage[peaks][::-1], wavelengths_nm[::-1]

    f = sp.interpolate.interp1d(x, y, kind='linear', fill_value='extrapolate')

    return f(voltage)

def piezo2wavelength_v2(voltage, freq=3):
    # returns the corresponding wavelength fot given Piezo voltage
    # by given freqency setting (1, 3, 4, 6). Default is freq=3
    #korrespondierende Wellenlängen
    wavelengths_nm = np.array([229, 233, 237, 241, 245, 249]) * 10**(-3) + 780

    laser = LaserData(f'Data/spektrum_dopplerfrei_ohne_pump_freq{freq}.dat')
    #starting time
    idx_start = np.argmax(laser.piezo_voltage)
    idx_stop  = np.argmin(laser.piezo_voltage)

    laser.slice_data(idx_start, idx_stop, filter=True)

    #filter data
    laser.piezo_voltage = savgol_filter(laser.piezo_voltage, 1000, 1)
    laser.fabry_perot = savgol_filter(laser.fabry_perot, 50, 6)

    peaks, _ = find_peaks(laser.fabry_perot,prominence=0.05,height=0.8)

    if freq==1:
        x, y = laser.piezo_voltage[peaks[1:6]][::-1], wavelengths_nm[1:6][::-1]
    else:
        x, y = laser.piezo_voltage[peaks[0:5]][::-1], wavelengths_nm[1:6][::-1]

    f = sp.interpolate.interp1d(x, y, kind='linear', fill_value='extrapolate')

    return f(voltage)


In [ ]:
v = np.linspace(-20, 20, 10)
print(piezo2wavelength(v)-780)

In [ ]:
#Alte Zuordnung
#relevante files einlesen
filenames = sorted([file for file in os.listdir('Data/') if file.__contains__('dopplerfrei_ohne_pump_freq')])

#new figure
%matplotlib inline
#plt.clf()
fig, axes = plt.subplots(2, 2, figsize=(11,6))

#Piezo-Spannungen für die händisch gemessenen Peaks (für absteigende Spannung)
peaks_manuell_V = np.array([-13.2, -8.36, -2.67, 2.96, 8.70, 14.6])[::-1]

#korrespondierende Wellenlängen
wavelengths_nm = np.array([229, 233, 237, 241, 245, 249]) * 10**(-3) + 780

for k, file in enumerate(filenames):
    laser = LaserData('Data/'+file)

    #starting time
    idx_start = np.argmax(laser.piezo_voltage)
    idx_stop  = np.argmin(laser.piezo_voltage)

    t_start = laser.time[idx_start]
    t_stop = laser.time[idx_stop]

    period = 2 * (t_stop - t_start)

    
    laser.slice_data(idx_start, idx_stop, filter=True)
    
    peaks,_ = find_peaks(laser.fabry_perot,prominence=0.05,height=0.8)
    
    ###################################plot (0, 0)#############################################
    ax = axes[0, 0]
    ax.plot(laser.piezo_voltage, laser.fabry_perot+k*1.4, label=f'T={period:.3f} s')
    for peak in peaks:
        ax.plot(laser.piezo_voltage[peak], laser.fabry_perot[peak]+k*1.4,'go')
    
    ###################################plot (0, 1)#############################################
    ax = axes[0, 1]
    if k==0:
        ax.scatter(laser.piezo_voltage[peaks[1:7]], wavelengths_nm-780, marker='x', label=f'T={period:.3f} s')
    else:
        ax.scatter(laser.piezo_voltage[peaks], wavelengths_nm-780, marker='x', label=f'T={period:.3f} s')
    ax.plot(laser.piezo_voltage, piezo2wavelength(laser.piezo_voltage, freq=int(file[-5]))-780, '--', linewidth=1)
    # v = np.linspace(-10, 10, 1000)
    # ax.plot(v, piezo2wavelength(v, freq=int(file[-5]))-780, '--', linewidth=1)

    ###################################plot (1, 0)#############################################
    ax = axes[1, 0]

    ax.plot(piezo2wavelength(laser.piezo_voltage, freq=int(file[-5]))-780, laser.fabry_perot+k*1.4, label=f'T={period:.3f} s')
    for peak in peaks:
        ax.plot(piezo2wavelength(laser.piezo_voltage[peak], freq=int(file[-5]))-780, laser.fabry_perot[peak]+k*1.4,'go')

    ###################################plot (1, 0)#############################################
    ax = axes[1, 1]
    ax.plot(piezo2wavelength(laser.piezo_voltage, freq=int(file[-5]))-780, laser.probe_laser+k*1.4, label=f'T={period:.3f} s')



ax = axes[0, 0]
for i, peak in enumerate(peaks_manuell_V):
        ax.axvline(peak, linestyle='--', color='black', linewidth=1)

ax = axes[0, 1]
ax.scatter(peaks_manuell_V, wavelengths_nm-780, marker='x', color='black', label='manuell')
f = sp.interpolate.interp1d(peaks_manuell_V, wavelengths_nm, kind='linear', fill_value='extrapolate')
ax.plot(laser.piezo_voltage, f(laser.piezo_voltage)-780, '--', color='black', linewidth=1)

ax = axes[1, 0]
for i, peak in enumerate(peaks_manuell_V):
        ax.axvline(wavelengths_nm[i]-780, linestyle='--', color='black', linewidth=1)

ax = axes[0, 0]
ax.set_xlabel('Piezospannung in V')
ax.set_ylabel('Fabry-Pérot-Spannung in V')


ax = axes[0, 1]
ax.set_xlabel('Piezospannung in V')
ax.set_ylabel(r'$(\lambda-780)$ in nm')
ax.legend(bbox_to_anchor=(1.01, 1.0), loc="upper left")

ax = axes[1, 0]
ax.set_xlabel(r'$(\lambda-780)$ in nm')
ax.set_ylabel('Fabry-Pérot-Spannung in V')

ax = axes[1, 1]
ax.set_xlabel(r'$(\lambda-780)$ in nm')
ax.set_ylabel('Probenlaser-Spannung in V')
ax.legend(bbox_to_anchor=(1.01, 1.0), loc="upper left")

plt.tight_layout()
plt.savefig('FP_kalibrierung.pdf')

In [ ]:
#Die Zuordnung ist verworfen
# #Andere Zuordnung
# #relevante files einlesen
# filenames = sorted([file for file in os.listdir('Data/') if file.__contains__('dopplerfrei_ohne_pump_freq')])

# #new figure
# %matplotlib inline
# #plt.clf()
# fig, axes = plt.subplots(2, 2, figsize=(11,6))

# #Piezo-Spannungen für die händisch gemessenen Peaks (für absteigende Spannung)
# peaks_manuell_V = np.array([-13.2, -8.36, -2.67, 2.96, 8.70, 14.6])[::-1]

# #korrespondierende Wellenlängen
# wavelengths_nm = np.array([229, 233, 237, 241, 245, 249]) * 10**(-3) + 780

# for k, file in enumerate(filenames):
#     laser = LaserData('Data/'+file)

#     #starting time
#     idx_start = np.argmax(laser.piezo_voltage)
#     idx_stop  = np.argmin(laser.piezo_voltage)

#     t_start = laser.time[idx_start]
#     t_stop = laser.time[idx_stop]

#     period = 2 * (t_stop - t_start)

    
#     laser.slice_data(idx_start, idx_stop, filter=True)
    
#     peaks,_ = find_peaks(laser.fabry_perot,prominence=0.05,height=0.8)
    
#     ###################################plot (0, 0)#############################################
#     ax = axes[0, 0]
#     ax.plot(laser.piezo_voltage, laser.fabry_perot+k*1.4)
#     for peak in peaks:
#         ax.plot(laser.piezo_voltage[peak], laser.fabry_perot[peak]+k*1.4,'mo')
    
#     ###################################plot (0, 1)#############################################
#     ax = axes[0, 1]
#     if k==0:
#         ax.scatter(laser.piezo_voltage[peaks[1:6]], wavelengths_nm[1:6]-780, marker='x', label=f'T={period:.3f} s')
#     else:
#         ax.scatter(laser.piezo_voltage[peaks[0:5]], wavelengths_nm[1:6]-780, marker='x', label=f'T={period:.3f} s')
#     ax.plot(laser.piezo_voltage, piezo2wavelength_v2(laser.piezo_voltage, freq=int(file[-5]))-780, '--', linewidth=1)

#     ###################################plot (1, 0)#############################################
#     ax = axes[1, 0]

#     ax.plot(piezo2wavelength_v2(laser.piezo_voltage)-780, laser.fabry_perot+k*1.4, label=f'T={period:.3f} s')
#     for peak in peaks:
#         ax.plot(piezo2wavelength_v2(laser.piezo_voltage[peak])-780, laser.fabry_perot[peak]+k*1.4,'mo')

#     ###################################plot (1, 0)#############################################
#     ax = axes[1, 1]
#     ax.plot(piezo2wavelength_v2(laser.piezo_voltage)-780, laser.probe_laser+k*1.4, label=f'T={period:.3f} s')



# ax = axes[0, 0]
# for i, peak in enumerate(peaks_manuell_V):
#         ax.axvline(peak, linestyle='--', color='black', linewidth=1)

# ax = axes[0, 1]
# ax.scatter(peaks_manuell_V, wavelengths_nm-780, marker='x', color='black', label='manuell')
# f = sp.interpolate.interp1d(peaks_manuell_V, wavelengths_nm, kind='linear', fill_value='extrapolate')
# ax.plot(laser.piezo_voltage, f(laser.piezo_voltage)-780, '--', color='black', linewidth=1)

# ax = axes[1, 0]
# for i, peak in enumerate(peaks_manuell_V):
#         ax.axvline(wavelengths_nm[i]-780, linestyle='--', color='black', linewidth=1)

# ax = axes[0, 0]
# ax.set_xlabel('Piezospannung in V')
# ax.set_ylabel('Fabry-Perot-Spannung in V')

# ax = axes[0, 1]
# ax.legend()
# ax.set_xlabel('Piezospannung in V')
# ax.set_ylabel(r'$(\lambda-780)$ in nm')

# ax = axes[1, 0]
# ax.set_xlabel(r'$(\lambda-780)$ in nm')
# ax.set_ylabel('Fabry-Perot-Spannung in V')

# ax = axes[1, 1]
# ax.set_xlabel(r'$(\lambda-780)$ in nm')
# ax.set_ylabel('Probenlaser-Spannung in V')
# ax.legend()

# plt.tight_layout()

# Dopplerfrei

Theoretisches Spektrum

In [ ]:
import scipy.constants as const
#Rb85
RB85_GHz = {
    'S1/2F2' : -1.7708439228,
    'S1/2F3' : 1.2648885163,
    'P1/2F2' : 377.107385690*1000 - 210.923*0.001,
    'P1/2F3' : 377.107385690*1000 + 150.659*0.001,
    'P3/2F1' : 384.230406373*1000 - 113.208*0.001,
    'P3/2F2' : 384.230406373*1000 - 83.835*0.001,
    'P3/2F3' : 384.230406373*1000 - 20.435*0.001,
    'P3/2F4' : 384.230406373*1000 + 100.205*0.001,
}


#Rb87
RB87_GHz = {
    'S1/2F1' : -4.271676631815181,
    'S1/2F2' : 2.563005979089109,
    'P1/2F1' : 377.107463380*1000 - 509.05*0.001,
    'P1/2F2' : 377.107463380*1000 + 305.43*0.001,
    'P3/2F0' : 384.2304844685*1000 - 302.0738*0.001,
    'P3/2F1' : 384.2304844685*1000 - 229.8518*0.001,
    'P3/2F2' : 384.2304844685*1000 - 72.9112*0.001,
    'P3/2F3' : 384.2304844685*1000 + 193.7407*0.001,
}

def transition_wavelength_nm(isotope, niveau1, niveau2):
    c = const.speed_of_light                    #m/s
    if isotope==85:
        freq1 = RB85_GHz[niveau1]
        freq2 = RB85_GHz[niveau2]
    if isotope==87:
        freq1 = RB87_GHz[niveau1]
        freq2 = RB87_GHz[niveau2]

    return  np.abs(c / (freq1 - freq2))

#für die D2 Linie direkt
def transition_wavelength_D2_nm(isotope, F_S, F_P):
    c = const.speed_of_light                    #m/s
    niveau1 = f'S1/2F{F_S}'
    niveau2 = f'P3/2F{F_P}'
    lambda_nm = transition_wavelength_nm(isotope, niveau1, niveau2)

    return  lambda_nm

#Wellenlängen für alle möglichen Übergänge
TRANSITION_WAVELENGTH_nm = {} #zum speichern für später eventuell
for iso in [85, 87]:
    #print(f'Rb{iso}')
    for j_P in ['1/2', '3/2']:
        for f_S in range(4):
            for delta_m in range(-1, 2):
                try:
                    f_P = f_S + delta_m #Auswahlregel
                    niveau1 = f'S1/2F{f_S}'
                    niveau2 = f'P{j_P}F{f_P}'
                    lambda_nm = transition_wavelength_nm(iso, niveau1, niveau2)
                    transition_key = f'{iso}{niveau1}<->{niveau2}'
                    TRANSITION_WAVELENGTH_nm[transition_key] = lambda_nm
                    #print(lambda_nm)
                except KeyError:
                    continue

pprint.pprint(TRANSITION_WAVELENGTH_nm)

In [ ]:
def theo_spektrum(lambda_nm, iso):
    #mittlere Lebensdauer und natürliches Mischungsverhältnis
    c = const.c
    if np.isscalar(lambda_nm):
        intensity = 0
    else:
        intensity = np.zeros(len(lambda_nm))

    if iso == 85:
        factor = 0.7217
        tau_ns = 26.2348
    if iso == 87:
        factor = 0.2783
        tau_ns = 26.24

    for transition, wl in TRANSITION_WAVELENGTH_nm.items():
        if (transition.__contains__(str(iso)) and transition.__contains__('P3/2')):
            delta_lambda_nm = wl**2 / (2*np.pi*c*tau_ns)
            contribution_peak = factor * (delta_lambda_nm / ((lambda_nm-wl)**2 + 0.25*delta_lambda_nm**2))
            intensity = intensity + contribution_peak
    return intensity    

#mittlere Peaks
peaks_theo = np.zeros(4)
for transition, wavelength in TRANSITION_WAVELENGTH_nm.items():
    if (transition.__contains__('85S1/2F2') and transition.__contains__('P3/2')):
        peaks_theo[0] += wavelength
    if (transition.__contains__('85S1/2F3') and transition.__contains__('P3/2')):
        peaks_theo[1] += wavelength
    if (transition.__contains__('87S1/2F1') and transition.__contains__('P3/2')):
        peaks_theo[2] += wavelength
    if (transition.__contains__('87S1/2F2') and transition.__contains__('P3/2')):
        peaks_theo[3] += wavelength

peaks_theo = np.sort(peaks_theo) / 3
print(peaks_theo)

In [ ]:
laser_mit_doppler = LaserData('Data/spektrum_dopplerfrei_ohne_pump_freq3.dat')
laser_ohne_doppler = LaserData('Data/spektrum_dopplerfrei_mit_pump_freq3.dat')


idx_start = np.argmax(laser_mit_doppler.piezo_voltage)
idx_stop  = np.argmin(laser_mit_doppler.piezo_voltage)
laser_mit_doppler.slice_data(idx_start, idx_stop, filter=True)

idx_start = np.argmax(laser_ohne_doppler.piezo_voltage)
idx_stop  = np.argmin(laser_ohne_doppler.piezo_voltage)
laser_ohne_doppler.slice_data(idx_start, idx_stop, filter=True)


#Zuordnung 1
wavelength_md = piezo2wavelength(laser_mit_doppler.piezo_voltage)
wavelength_od = piezo2wavelength(laser_ohne_doppler.piezo_voltage)

%matplotlib widget
fig, axes =  plt.subplots(2, figsize=(11, 9))

ax = axes[0]
ax.plot(wavelength_md-780, laser_mit_doppler.probe_laser, label='ohne Pumplaser')
ax.plot(wavelength_od-780, laser_ohne_doppler.probe_laser, label='mit Pumplaser')

diff = laser_ohne_doppler.probe_laser - np.interp(wavelength_od, wavelength_md, laser_mit_doppler.probe_laser)
normalize_data = np.max(diff)
ax = axes[1]



#Zuordnung 2
# wavelength_md = piezo2wavelength_v2(laser_mit_doppler.piezo_voltage)
# wavelength_od = piezo2wavelength_v2(laser_ohne_doppler.piezo_voltage)

# ax = axes[0]
# ax.plot(wavelength_md-780, laser_mit_doppler.probe_laser, label='Zuordnung 2')
# ax.plot(wavelength_od-780, laser_ohne_doppler.probe_laser, label='Zuordnung 2')



# diff = laser_ohne_doppler.probe_laser - np.interp(wavelength_od, wavelength_md, laser_mit_doppler.probe_laser)
# normalize_data = np.max(diff)
# ax = axes[1]
# ax.plot(wavelength_od-780, diff/normalize_data, label='Zuordnung 2')

#theoretisches Spektrum
# for transition, wl in TRANSITION_WAVELENGTH_nm.items():
#     if transition.__contains__('P3/2'):
#         if transition.__contains__('85'):
#             ax.axvline(wl-780, color='red')
#         if transition.__contains__('87'):
#             ax.axvline(wl-780, color='green')
#     else:
#         continue

normalize_theo = np.max(theo_spektrum(wavelength_od, 85))

wl_theo = np.linspace(780.227, 780.250, 100*len(wavelength_od))

ax = axes[1]
ax.plot(wl_theo-780, theo_spektrum(wl_theo, 85)/normalize_theo, color='red', label='Rb85')
ax.plot(wl_theo-780, theo_spektrum(wl_theo, 87)/normalize_theo, color='green', label='Rb87')
ax.plot(wavelength_od-780, diff/normalize_data, label='Messdaten')


ax = axes[0]
ax.set_ylabel('Intensität (bel. Einheiten)')
ax.legend()

ax = axes[1]
ax.legend()
ax.set_ylabel('Intensität (bel. Einheiten)')
ax.set_xlabel(r'$(\lambda-780)$ in nm')

# ax = axes[2]
# ax.legend()
plt.tight_layout()
plt.savefig('dopplerfrei_ungefiltered.pdf')

In [ ]:
from scipy.signal import butter, lfilter, freqz
from matplotlib.gridspec import GridSpec
#Periodische Modulation rund 85 Hz
%matplotlib widget
plt.clf()

s2pm = 8.463 #pm/s
signal = diff/normalize_data
wavelength = wavelength_od

wl_theo = np.linspace(780.227, 780.2478, 100*len(wavelength_od))

spectrum_theo85 = theo_spektrum(wl_theo, 85)/normalize_theo
spectrum_theo87 = theo_spektrum(wl_theo, 87)/normalize_theo


fig = plt.figure(figsize=(11, 9))
gs = GridSpec(3, 2, figure=fig)
ax0 = fig.add_subplot(gs[0, :])
ax0.plot(wavelength[0:90000]-780, signal[0:90000], label='Messdaten')
ax0.plot(wl_theo-780, spectrum_theo85+spectrum_theo87, label='theoretisches Spektrum')
peaks,_ = find_peaks(signal[0:90000],prominence=0.2,height=0.2)
print(peaks)
ax0.plot(wavelength[peaks]-780, signal[peaks], 'go')

ax0.set_xlabel(r'$(\lambda-780)$ in nm')
ax0.set_ylabel('Intensität (bel. Einheiten)')
ax0.legend()

ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(wavelength-780, signal, label='Messdaten gefiltert')
ax1.plot(wavelength[peaks]-780, signal[peaks], 'go')
xmin, xmax = 0.2296, 0.2314
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(-0.04, 0.5)
ax1.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))
ax1.set_ylabel('Intensität (bel. Einheiten)')

ax2 = fig.add_subplot(gs[1, 1])
ax2.plot(wavelength-780, signal, label='Messdaten gefiltert')
ax2.plot(wavelength[peaks]-780, signal[peaks], 'go')
xmin, xmax = 0.2344, 0.2365
ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-0.04, 1.05)
ax2.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))

ax3 = fig.add_subplot(gs[2, 1])
ax3.plot(wavelength-780, signal, label='Messdaten gefiltert')
ax3.plot(wavelength[peaks]-780, signal[peaks], 'go')
xmin, xmax = 0.2405, 0.2423
ax3.set_xlim(xmin, xmax)
ax3.set_ylim(-0.04, 1)
ax3.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))
ax3.set_xlabel(r'$(\lambda-780)$ in nm')

ax4 = fig.add_subplot(gs[2, 0])
ax4.plot(wavelength-780, signal, label='Messdaten gefiltert')
ax4.plot(wavelength[peaks]-780, signal[peaks], 'go')
xmin, xmax = 0.2425, 0.2446
ax4.set_xlim(xmin, xmax)
ax4.set_ylim(-0.04, 0.5)
ax4.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))
ax4.set_xlabel(r'$(\lambda-780)$ in nm')
ax4.set_ylabel('Intensität (bel. Einheiten)')

plt.tight_layout()
plt.savefig('ungefiltert_peaks.pdf')
plt.show()

In [ ]:
from scipy.signal import butter, lfilter, freqz
from matplotlib.gridspec import GridSpec
#Periodische Modulation rund 85 Hz
%matplotlib widget
plt.clf()

s2pm = 8.463 #pm/s
signal = diff/normalize_data
wavelength = wavelength_od

wl_range_pm = (np.max(wavelength) - np.min(wavelength)) * 1000
fs = s2pm / (wl_range_pm/len(wavelength))
print(wl_range_pm/len(wavelength))
order = 6
cutoff = 40 #Hz

b, a = butter(order, cutoff, fs=fs, btype='low', analog=False)
signal_filtered = lfilter(b, a, signal)

wl_theo = np.linspace(780.227, 780.2478, 100*len(wavelength_od))

spectrum_theo85 = theo_spektrum(wl_theo, 85)/normalize_theo
spectrum_theo87 = theo_spektrum(wl_theo, 87)/normalize_theo




fig = plt.figure(figsize=(11, 9))
gs = GridSpec(3, 2, figure=fig)
ax0 = fig.add_subplot(gs[0, :])
ax0.plot(wavelength[0:90000]-780, signal_filtered[0:90000], label='Messdaten gefiltert')
ax0.plot(wl_theo-780, spectrum_theo85+spectrum_theo87, label='theoretisches Spektrum')
peaks,_ = find_peaks(signal_filtered[0:90000],prominence=0.05,height=0.03)
peaks_exp = wavelength[peaks]
ax0.plot(wavelength[peaks]-780, signal_filtered[peaks], 'go')

for i, peak in enumerate(peaks_theo):
        ax0.axvline(peak-780, linestyle='--', color='black', linewidth=1)

ax0.set_xlabel(r'$(\lambda-780)$ in nm')
ax0.set_ylabel('Intensität (bel. Einheiten)')
ax0.legend()

ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(wavelength-780, signal_filtered, label='Messdaten gefiltert')
ax1.plot(wavelength[peaks]-780, signal_filtered[peaks], 'go')
xmin, xmax = 0.2296, 0.2314
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(-0.04, 0.44)
ax1.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))
ax1.set_ylabel('Intensität (bel. Einheiten)')

ax2 = fig.add_subplot(gs[1, 1])
ax2.plot(wavelength-780, signal_filtered, label='Messdaten gefiltert')
ax2.plot(wavelength[peaks]-780, signal_filtered[peaks], 'go')
xmin, xmax = 0.2344, 0.2365
ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-0.04, 1)
ax2.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))

ax3 = fig.add_subplot(gs[2, 1])
ax3.plot(wavelength-780, signal_filtered, label='Messdaten gefiltert')
ax3.plot(wavelength[peaks]-780, signal_filtered[peaks], 'go')
xmin, xmax = 0.2405, 0.2423
ax3.set_xlim(xmin, xmax)
ax3.set_ylim(-0.04, 1)
ax3.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))
ax3.set_xlabel(r'$(\lambda-780)$ in nm')

ax4 = fig.add_subplot(gs[2, 0])
ax4.plot(wavelength-780, signal_filtered, label='Messdaten gefiltert')
ax4.plot(wavelength[peaks]-780, signal_filtered[peaks], 'go')
xmin, xmax = 0.2425, 0.2446
ax4.set_xlim(xmin, xmax)
ax4.set_ylim(-0.04, 0.44)
ax4.set_xticks(np.linspace(xmin, xmax, 4, endpoint=True))
ax4.set_xlabel(r'$(\lambda-780)$ in nm')
ax4.set_ylabel('Intensität (bel. Einheiten)')


plt.tight_layout()
plt.savefig('dopplerfrei_gefiltert.pdf')

In [ ]:
#Verhältnis Rb85 / Rb87
height_87 = np.array([0.377, 0.373])
height_85 = np.array([0.962, 0.81])
H87 = np.mean(height_87)
H85 = np.mean(height_85)
print('exp.    :', H87/H85)
print('theo.   :', 0.2783/0.7217)
print('exp/theo:', H87/H85 / (0.2783/0.7217))

In [ ]:
#Peakabweichungen
print('absolute Abweichungen')
print(peaks_exp/peaks_theo)

diffs_theo = np.diff(peaks_theo)
diffs_exp = np.diff(peaks_exp)

print('relative Abweichungen')
print(diffs_exp/diffs_theo)